In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers

In [2]:
class VAE_MNIST:
    
    def __init__(self, dim_z, kl_weight, learning_rate):
        # change dim from (28, 28, 1)
        self.dim_x = (32, 32, 1)
        self.dim_z = dim_z
        self.kl_weight = kl_weight
        self.learning_rate = learning_rate

    # Sequential API encoder
    def encoder_z(self):
        # define prior distribution for the code, which is an isotropic Gaussian
        prior = tfd.Independent(tfd.Normal(loc=tf.zeros(self.dim_z), scale=1.), 
                                reinterpreted_batch_ndims=1)
        # build layers argument for tfk.Sequential()
        input_shape = self.dim_x
        layers = [tfkl.InputLayer(input_shape=input_shape)]
        layers.append(tfkl.Conv2D(filters=32, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        layers.append(tfkl.Conv2D(filters=64, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        layers.append(tfkl.Flatten())
        # the following two lines set the output to be a probabilistic distribution
        layers.append(tfkl.Dense(tfpl.IndependentNormal.params_size(self.dim_z), 
                                 activation=None, name='z_params'))
        layers.append(tfpl.IndependentNormal(self.dim_z, 
            convert_to_tensor_fn=tfd.Distribution.sample, 
            activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=self.kl_weight), 
            name='z_layer'))
        return tfk.Sequential(layers, name='encoder')
    
    # Sequential API decoder
    def decoder_x(self):
        layers = [tfkl.InputLayer(input_shape=self.dim_z)]
        # probably 7 before since 28/2/2 = 7, so changing to 32/2/2 = 8
        layers.append(tfkl.Dense(8*8*32, activation=None))
        layers.append(tfkl.Reshape((8,8,32)))
        layers.append(tfkl.Conv2DTranspose(filters=64, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        layers.append(tfkl.Conv2DTranspose(filters=32, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        layers.append(tfkl.Conv2DTranspose(filters=1, kernel_size=3, strides=1, 
                                           padding='same'))
        layers.append(tfkl.Flatten(name='x_params'))
        # note that here we don't need 
        # `tfkl.Dense(tfpl.IndependentBernoulli.params_size(self.dim_x))` because 
        # we've restored the desired input shape with the last Conv2DTranspose layer
        layers.append(tfpl.IndependentBernoulli(self.dim_x, name='x_layer'))
        return tfk.Sequential(layers, name='decoder')
    
    def build_vae_keras_model(self):
        x_input = tfk.Input(shape=self.dim_x)
        encoder = self.encoder_z()
        decoder = self.decoder_x()
        z = encoder(x_input)

        # compile VAE model
        model = tfk.Model(inputs=x_input, outputs=decoder(z))
        model.compile(loss=negative_log_likelihood, 
                      optimizer=tfk.optimizers.Adam(self.learning_rate))
        return model

# the negative of log-likelihood for probabilistic output
negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

In [3]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
print(tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

import deepcell
# Changed from before due to new placement of Track, concat_tracks
from deepcell_tracking.utils import load_trks
from deepcell.data.tracking import Track, concat_tracks
##############
from sklearn.model_selection import train_test_split
from deepcell.utils.data_utils import reshape_movie
from deepcell.utils.transform_utils import erode_edges
from deepcell.data import split_dataset
from deepcell_toolbox.processing import normalize, histogram_normalization

import spektral

2.8.0


In [4]:
# Let's create a binary dataset on which to train the model. It should be of the same format as the above dataset

In [5]:
vae = VAE_MNIST(1024, 1, 1e-3)

In [39]:
# There is a real problem with somewhat large data (wasn't able to allocate space)

train_data = np.random.choice([0, 1], size=(3000, 1, 32, 32, 1), p=[0.5, 0.5])
val_data = np.random.choice([0, 1], size=(1000, 1, 32, 32, 1), p=[0.5, 0.5])

train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_data))
train_ds.batch(10)
val_ds = tf.data.Dataset.from_tensor_slices((val_data, val_data))
val_ds.batch(10)

<BatchDataset element_spec=(TensorSpec(shape=(None, 1, 32, 32, 1), dtype=tf.int64, name=None), TensorSpec(shape=(None, 1, 32, 32, 1), dtype=tf.int64, name=None))>

In [40]:
AE = vae.build_vae_keras_model()

In [41]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [42]:
# steps_per_epoch = 2048
# validation_steps = 100
n_epochs = 1

# train_callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(
#         model_path, monitor='val_loss',
#         save_best_only=True, verbose=1,
#         save_weights_only=False),
#     tf.keras.callbacks.ReduceLROnPlateau(
#         monitor='val_loss', factor=0.5, verbose=1,
#         patience=3, min_lr=1e-7)
# ]

loss_history = AE.fit(
    train_ds,
#    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
#    validation_steps=validation_steps,
    epochs=n_epochs,
    verbose=1,
#    callbacks=train_callbacks
)

2022-07-15 22:43:57.573872: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


3000/3000 [==============================] - 28s 9ms/step - loss: 721.2941 - val_loss: 717.6370
